In [2]:
from rdflib import ConjunctiveGraph, URIRef, Literal, Namespace, Dataset
SCHEMA = Namespace("http://schema.org/")

## 1. Creating named graphs from scratch

In [3]:
G = ConjunctiveGraph()
G.add((URIRef("http://test1"), SCHEMA.name, Literal("name1"), URIRef("http://univ-nantes.fr/itx") ))
G.add((URIRef("http://test2"), SCHEMA.name, Literal("name2"), URIRef("http://univ-nantes.fr/itx") ))
G.add((URIRef("http://test3"), SCHEMA.name, Literal("name3"), URIRef("http://amu.fr/tagc") ))

<Graph identifier=Ndd784213a5bf4110892a005234894b3b (<class 'rdflib.graph.ConjunctiveGraph'>)>

In [4]:
print(G.serialize(format="trig"))
print(G.serialize(format="nquads"))

@prefix ns1: <http://univ-nantes.fr/> .
@prefix ns2: <http://schema.org/> .
@prefix ns3: <http://amu.fr/> .

ns1:itx {
    <http://test1> ns2:name "name1" .

    <http://test2> ns2:name "name2" .
}

ns3:tagc {
    <http://test3> ns2:name "name3" .
}


<http://test1> <http://schema.org/name> "name1" <http://univ-nantes.fr/itx> .
<http://test2> <http://schema.org/name> "name2" <http://univ-nantes.fr/itx> .
<http://test3> <http://schema.org/name> "name3" <http://amu.fr/tagc> .




In [5]:
print(G.serialize(format="turtle"))

@prefix ns2: <http://schema.org/> .

<http://test1> ns2:name "name1" .

<http://test2> ns2:name "name2" .

<http://test3> ns2:name "name3" .




In [6]:
G2 = ConjunctiveGraph(identifier="http://univ-nantes.fr/itx")
G2.bind("sc", SCHEMA)
G2.bind("unantes", Namespace("http://univ-nantes.fr/"))
G2.add((URIRef("http://test1"), SCHEMA.name, Literal("name1")))
G2.add((URIRef("http://test2"), SCHEMA.name, Literal("name2")))

G3 = ConjunctiveGraph(identifier="http://amu/tagc")
G3.bind("amu", Namespace("http://amu/"))
G3.bind("sc", SCHEMA)
G3.add((URIRef("http://test3"), SCHEMA.name, Literal("name3")))
G3.add((URIRef("http://test4"), SCHEMA.name, Literal("name4")))
G3.add((URIRef("http://test4"), SCHEMA.identifier, Literal("id4")))

ds = Dataset()
ds.add_graph(G2)
ds.add_graph(G3)


print(ds.serialize(format="trig"))
print(ds.serialize(format="nquads"))

@prefix amu: <http://amu/> .
@prefix ns1: <urn:x-rdflib:> .
@prefix sc: <http://schema.org/> .
@prefix unantes: <http://univ-nantes.fr/> .

unantes:itx {
    <http://test1> sc:name "name1" .

    <http://test2> sc:name "name2" .
}

amu:tagc {
    <http://test3> sc:name "name3" .

    <http://test4> sc:identifier "id4" ;
        sc:name "name4" .
}


<http://test1> <http://schema.org/name> "name1" <http://univ-nantes.fr/itx> .
<http://test2> <http://schema.org/name> "name2" <http://univ-nantes.fr/itx> .
<http://test4> <http://schema.org/identifier> "id4" <http://amu/tagc> .
<http://test4> <http://schema.org/name> "name4" <http://amu/tagc> .
<http://test3> <http://schema.org/name> "name3" <http://amu/tagc> .




## 2. Querying named graphs

In [7]:
q1 = """
PREFIX schema: <http://schema.org/>

SELECT * WHERE {
    GRAPH ?g {
        ?s schema:identifier ?id
    }
}
"""

In [8]:
res = ds.query(q1)
for r in res:
    print(f" found ID {r['id']} for resource {r['s']} originating from {r['g']}")

 found ID id4 for resource http://test4 originating from http://amu/tagc


In [9]:
q2 = """
PREFIX schema: <http://schema.org/>

SELECT * WHERE {
    GRAPH ?g {
        ?s schema:name ?name
    }
}
"""

In [10]:
res = ds.query(q2)
for r in res:
    print(f" found name {r['name']} for resource {r['s']} originating from {r['g']}")

 found name name1 for resource http://test1 originating from http://univ-nantes.fr/itx
 found name name2 for resource http://test2 originating from http://univ-nantes.fr/itx
 found name name3 for resource http://test3 originating from http://amu/tagc
 found name name4 for resource http://test4 originating from http://amu/tagc


## Validating SHACL shapes on named graphs

In [41]:
trig = """
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix amu: <http://amu/> .
@prefix ns1: <urn:x-rdflib:> .
@prefix sc: <http://schema.org/> .
@prefix scs: <https://schema.org/> .
@prefix unantes: <http://univ-nantes.fr/> .

unantes:itx {
    <http://test1>  rdf:type scs:Person ;
                    sc:name "name1" .

    <http://test2>  rdf:type scs:Person ; 
                    sc:name "name2" .
}

amu:tagc {
    <http://test3>  rdf:type scs:Person ; 
                    scs:name "name3" .

    <http://test4>  rdf:type scs:Person ; 
                    sc:identifier "id4" .
}

"""

kg = ConjunctiveGraph()
kg.parse(data=trig, format="trig")
print(len(kg))
print(kg.serialize(format="nquads"))

8
<http://test1> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://schema.org/Person> <http://univ-nantes.fr/itx> .
<http://test1> <http://schema.org/name> "name1" <http://univ-nantes.fr/itx> .
<http://test2> <http://schema.org/name> "name2" <http://univ-nantes.fr/itx> .
<http://test2> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://schema.org/Person> <http://univ-nantes.fr/itx> .
<http://test4> <http://schema.org/identifier> "id4" <http://amu/tagc> .
<http://test4> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://schema.org/Person> <http://amu/tagc> .
<http://test3> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://schema.org/Person> <http://amu/tagc> .
<http://test3> <https://schema.org/name> "name3" <http://amu/tagc> .




In [44]:
trig2 = """
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix amu: <http://amu/> .
@prefix urennes: <http://urennes/> .
@prefix ns1: <urn:x-rdflib:> .
@prefix sc: <http://schema.org/> .
@prefix unantes: <http://univ-nantes.fr/> .

urennes:irisa {
    <http://test5>  rdf:type sc:Person ;
                    sc:name "name5" .
}
"""

kg.parse(data=trig2, format="trig")
print(len(kg))
print(kg.serialize(format="nquads"))

11
<http://test4> <http://schema.org/identifier> "id4" <http://amu/tagc> .
<http://test3> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Person> <http://amu/tagc> .
<http://test4> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Person> <http://amu/tagc> .
<http://test3> <http://schema.org/name> "name3" <http://amu/tagc> .
<http://test5> <http://schema.org/name> "name5" <http://urennes/irisa> .
<http://test5> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Person> <http://urennes/irisa> .
<http://test4> <http://schema.org/name> "name5" <http://urennes/irisa> .
<http://test4> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Person> <http://urennes/irisa> .
<http://test2> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Person> <http://univ-nantes.fr/itx> .
<http://test2> <http://schema.org/name> "name2" <http://univ-nantes.fr/itx> .
<http://test1> <http://schema.org/name> "name1" <http

In [13]:
for (s, p, o, g) in kg.quads():
    print(type(g))
    print(f"{s}, {p}, {o}, {g.identifier}")

<class 'rdflib.graph.Graph'>
http://test4, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://schema.org/Person, http://amu/tagc
<class 'rdflib.graph.Graph'>
http://test2, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://schema.org/Person, http://univ-nantes.fr/itx
<class 'rdflib.graph.Graph'>
http://test1, http://schema.org/name, name1, http://univ-nantes.fr/itx
<class 'rdflib.graph.Graph'>
http://test2, http://schema.org/name, name2, http://univ-nantes.fr/itx
<class 'rdflib.graph.Graph'>
http://test3, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://schema.org/Person, http://amu/tagc
<class 'rdflib.graph.Graph'>
http://test4, http://schema.org/identifier, id4, http://amu/tagc
<class 'rdflib.graph.Graph'>
http://test1, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://schema.org/Person, http://univ-nantes.fr/itx
<class 'rdflib.graph.Graph'>
http://test3, https://schema.org/name, name3, http://amu/tagc


In [37]:
from jinja2 import Template

shape_tpl = """
    @prefix ns: <https://fair-checker.france-bioinformatique.fr#> .
    @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix sc: <http://schema.org/> .
    @prefix scs: <https://schema.org/> .
    @prefix bsc: <https://bioschemas.org/> .
    @prefix dct: <http://purl.org/dc/terms/> .
    @prefix sh: <http://www.w3.org/ns/shacl#> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
    @prefix edam: <http://edamontology.org/> .
    @prefix biotools: <https://bio.tools/ontology/> .

    ns:test_NG_shape
        a sh:NodeShape ;
        sh:targetClass sc:Person, scs:Person ;

        {% for min_prop in min_props %}
                sh:property [
                    {% if ("sc:" in min_prop) %}
                    sh:path [sh:alternativePath({{min_prop}} {{min_prop.replace("sc", "scs")}})] ;
                    {% else %}
                    sh:path {{min_prop}} ;
                    {% endif %}
                    sh:minCount 1 ;
                    sh:severity sh:Violation
                ] ;
        {% endfor %}
    .
"""

template = Template(shape_tpl)
shape = template.render(
    shape_name='name',
    min_props=["sc:name"]
)
print(shape)


    @prefix ns: <https://fair-checker.france-bioinformatique.fr#> .
    @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix sc: <http://schema.org/> .
    @prefix scs: <https://schema.org/> .
    @prefix bsc: <https://bioschemas.org/> .
    @prefix dct: <http://purl.org/dc/terms/> .
    @prefix sh: <http://www.w3.org/ns/shacl#> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
    @prefix edam: <http://edamontology.org/> .
    @prefix biotools: <https://bio.tools/ontology/> .

    ns:test_NG_shape
        a sh:NodeShape ;
        sh:targetClass sc:Person, scs:Person ;

        
                sh:property [
                    
                    sh:path [sh:alternativePath(sc:name scs:name)] ;
                    
                    sh:minCount 1 ;
                    sh:severity sh:Violation
                ] ;
        
    .


In [38]:
shape_v2 = """
    @prefix ns: <https://fair-checker.france-bioinformatique.fr#> .
    @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
    @prefix sc: <http://schema.org/> .
    @prefix scs: <https://schema.org/> .
    @prefix bsc: <https://bioschemas.org/> .
    @prefix dct: <http://purl.org/dc/terms/> .
    @prefix sh: <http://www.w3.org/ns/shacl#> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
    @prefix edam: <http://edamontology.org/> .
    @prefix biotools: <https://bio.tools/ontology/> .

    ns:test_NG_shape
        a sh:NodeShape ;
        sh:targetClass sc:Person, scs:Person ;

        sh:property [
            sh:path [sh:alternativePath(sc:name scs:name)] ;
            sh:minCount 1 ;
            sh:severity sh:Violation
        ] 
    .
"""

#[sh: alternativePath(ex:father ex: mother  )]

In [42]:
from pyshacl import validate

r = validate(
        data_graph=kg,
        data_graph_format="trig",
        shacl_graph=shape_v2,
        shacl_graph_format="turtle",
        ont_graph=None,
        inference="rdfs",
        abort_on_first=False,
        meta_shacl=False,
        debug=False,
)

conforms, results_graph, results_text = r

In [43]:
print(conforms)

False


In [44]:
print(results_text)

Validation Report
Conforms: False
Results (1):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path [ sh:alternativePath ( sc:name scs:name ) ] ; sh:severity sh:Violation ]
	Focus Node: <http://test4>
	Result Path: [ sh:alternativePath ( sc:name scs:name ) ]
	Message: Less than 1 values on <http://test4>->[ sh:alternativePath ( sc:name scs:name ) ]

